# AiDM Assignment 2
##  Locality Sensitive Hashing for Netflix Data

###  Load Data
First we read in the data

In [6]:
import numpy as np
data_origin = np.load('../user_movie.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../user_movie.npy'

Inthe interest of memory conservation (only stoing non-zero elements) and efficient rearrangement, we store the data as ones in indicies according to their user ID (column) and rated movies (rows, 1 if rated 0 if not) 

In [7]:
from scipy.sparse import csc_matrix
users_movies = csc_matrix((np.ones(data_origin.size), (data_origin[:,0], data_origin[:,1])), shape=(3, 3)).toarray()

NameError: name 'data_origin' is not defined

###  Matrix Definition
Next we define general parameters (so they are in one place and easy to alter), including the length of signatures, the number of bands, and the numbers of rows (ie, the number of signature permutations)

In [4]:
signature_length = 100
bands = 5
rows = 100
print('Each of the', bands, 'bands will contain', int(rows/bands), 'signature permutations')

Each of the 5 bands will contain 20 signature permutations
